<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-13 14:25:57
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.27 C
-------------------
Today PnL: 23.77 K (0.19%)
Current PnL: -19.08 L (-13.53%)
CY Booked + Current PnL: -16.21 L (-11.5%)
-------------------
Total profit:  3.72 L
Total loss:  -22.80 L
-------------------
Total Booked + Current PnL: 10.77 L (9.44%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.27%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 80.96 L (63.96%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.43 C (112.62%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.58,X-MC,91.49,140590.0,25812.0,9912.0,0.48,22.49,7.05,31.13,215.0,2.60,1.15,36.95,AR,ATH,TYRES
56,PIDILITIND,2504.06,3576.00,-16.23,X-LC,53.19,92880.0,17758.0,14396.0,0.63,23.64,15.50,42.81,87.0,1.23,0.76,16.48,X40,BTT,CHEMICALS
69,SBILIFE,1496.49,1924.99,-1.02,H-LC,82.98,146975.0,21270.0,14727.0,-0.10,16.92,10.02,28.63,116.0,1.44,1.20,30.94,AR,ATH,INSURANCE
70,SBIN,760.30,863.00,-13.45,M-LC,58.51,212437.0,10958.0,16251.0,2.87,5.44,7.65,13.51,91.0,0.67,1.73,16.38,XY25,NTT,BANKS
25,DMART,3484.00,5112.00,-20.84,X-LC,13.83,97428.0,13812.0,25263.0,0.89,16.52,25.93,46.73,123.0,0.55,0.79,19.26,AR,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TANLA,917.30,1963.11,-41.41,H-SC,95.74,143302.0,-96113.0,369074.0,11.04,-40.15,257.55,114.01,29.0,-0.26,1.17,31.43,AR,ATH,IT
88,VALIANTORG,512.64,838.00,-303.99,H-SC,5.32,86710.0,-46576.0,131166.0,5.17,-34.94,151.27,63.47,83.0,-0.36,0.71,45.35,XR,NTT,CHEMICALS
29,GREENPANEL,375.16,537.00,164.21,M-SC,39.89,117915.0,-65163.0,144139.0,4.24,-35.59,122.24,43.14,98.0,-0.45,0.96,10.92,XY24,NTT,MISC
76,STARHEALTH,551.56,761.00,-13.64,M-SC,41.49,141225.0,-58991.0,135011.0,4.19,-29.46,95.60,37.97,46.0,-0.44,1.15,13.39,XY24,NTT,INSURANCE
63,REPCOHOME,515.07,880.00,-56.78,H-SC,77.66,160056.0,-44942.0,190179.0,3.33,-21.92,118.82,70.85,130.0,-0.24,1.31,29.20,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1354.50,1800.00,-39.16,H-MC,59.57,44944.0,-32262.0,57654.0,-3.63,-41.79,128.28,32.89,41.0,-0.56,0.37,23.77,XR,NTT,BANKS
40,INFY,1432.81,2275.00,-12.42,X-LC,80.32,218466.0,19305.0,97764.0,-3.39,9.69,44.75,58.78,9.0,0.20,1.78,14.06,X40,BTT,IT
81,TCS,3928.29,4998.00,-15.84,X-LC,75.53,189788.0,-22340.0,80110.0,-2.92,-10.53,42.21,27.23,8.0,-0.28,1.55,8.76,X40,BTT,IT
30,HAPPSTMNDS,919.80,1491.84,-26.03,M-SC,55.32,62370.0,-34209.0,94272.0,-2.58,-35.42,151.15,62.19,22.0,-0.36,0.51,9.36,AR,ATH,IT
19,CERA,8421.60,10420.45,-18.62,X-SC,90.43,84045.0,-25436.0,51419.0,-2.26,-23.23,61.18,23.73,122.0,-0.49,0.69,27.20,X40N,ATH,CERAMICS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TATAMOTORS,776.02,1065.00,-53.26,M-LC,87.23,187024.0,-18621.0,95195.0,-2.09,-9.06,50.90,37.24,1.0,-0.20,1.53,21.73,XY24,NTT,AUTO
2,ABB,5319.37,8953.12,-38.41,H-LC,74.47,140875.0,7891.0,82947.0,0.86,5.93,58.88,68.31,2.0,0.10,1.15,14.87,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-26.39,H-LC,43.62,121995.0,-1008.0,63706.0,-0.40,-0.82,52.22,50.98,3.0,-0.02,1.00,11.46,AR,ATH,REALTY
10,AWL,342.88,485.00,-69.16,H-MC,4.26,166783.0,-50946.0,141198.0,-0.11,-23.40,84.66,41.45,5.0,-0.36,1.36,11.41,XY24,NTT,FMCG
92,WHIRLPOOL,1151.42,2270.00,-36.55,M-SC,88.30,159931.0,13701.0,128361.0,-0.94,9.37,80.26,97.15,7.0,0.11,1.30,36.63,X40,NTT,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.58,X-MC,91.49,140590.0,25812.0,9912.0,0.48,22.49,7.05,31.13,215.0,2.60,1.15,36.95,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,6.76,X-MC,15.96,188085.0,32650.0,37786.0,0.84,21.01,20.09,45.32,198.0,0.86,1.53,27.98,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-16.23,X-LC,53.19,92880.0,17758.0,14396.0,0.63,23.64,15.50,42.81,87.0,1.23,0.76,16.48,X40,BTT,CHEMICALS
57,POLYCAB,5005.92,7541.35,7.69,H-MC,92.55,168350.0,28184.0,42811.0,0.33,20.11,25.43,50.65,54.0,0.66,1.37,28.48,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-2.94,M-LC,86.17,187512.0,36306.0,29364.0,0.48,24.01,15.66,43.43,37.0,1.24,1.53,29.97,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,POLYCAB,5005.92,7541.35,7.69,H-MC,92.55,168350.0,28184.0,42811.0,0.33,20.11,25.43,50.65,54.0,0.66,1.37,28.48,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,-1.14,H-SC,67.02,160856.0,9492.0,36289.0,-0.49,6.27,22.56,30.25,131.0,0.26,1.31,27.31,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-2.94,M-LC,86.17,187512.0,36306.0,29364.0,0.48,24.01,15.66,43.43,37.0,1.24,1.53,29.97,XY24,NTT,MISC
93,WIPRO,243.46,420.00,-8.80,M-LC,51.06,156035.0,5090.0,104372.0,-2.18,3.37,66.89,72.51,58.0,0.05,1.27,17.48,XR,NTT,IT
70,SBIN,760.30,863.00,-13.45,M-LC,58.51,212437.0,10958.0,16251.0,2.87,5.44,7.65,13.51,91.0,0.67,1.73,16.38,XY25,NTT,BANKS


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-8.80,M-LC,51.06,156035.0,5090.0,104372.0,-2.18,3.37,66.89,72.51,58.0,0.05,1.27,17.48,XR,NTT,IT
37,INDIAMART,2311.97,4911.36,-52.77,H-SC,76.60,102018.0,291.0,114077.0,1.11,0.29,111.82,112.43,NaN,0.00,0.83,20.41,AR,ATH,MISC
12,BANKINDIA,113.49,190.00,-34.31,M-MC,39.89,128571.0,-581.0,87647.0,2.62,-0.45,68.17,67.42,71.0,-0.01,1.05,25.05,XR,NTT,BANKS
91,VOLTAS,1272.00,1929.20,-30.74,H-MC,17.02,138600.0,-1320.0,73610.0,-0.45,-0.94,53.11,51.67,19.0,-0.02,1.13,4.69,AR,ATH,AC
18,CAMS,3643.00,5250.99,-10.00,H-SC,7.45,101195.0,-809.0,45831.0,-1.57,-0.79,45.29,44.14,47.0,-0.02,0.83,17.32,XR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,52.77,60.00,36.52,M-SC,81.91,118746.0,-23733.0,43259.0,2.18,-16.66,36.43,13.70,219.0,-0.55,0.97,40.15,X40N,NTT,BANKS
53,MRF,114777.60,150504.14,14.58,X-MC,91.49,140590.0,25812.0,9912.0,0.48,22.49,7.05,31.13,215.0,2.60,1.15,36.95,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-1.02,H-LC,82.98,146975.0,21270.0,14727.0,-0.10,16.92,10.02,28.63,116.0,1.44,1.20,30.94,AR,ATH,INSURANCE
86,UNIONBANK,123.87,163.00,-15.64,M-LC,68.09,148833.0,7993.0,36494.0,2.18,5.68,24.52,31.59,83.0,0.22,1.21,29.28,XY24,NTT,BANKS
52,MEDANTA,1022.60,1486.00,6.76,X-MC,15.96,188085.0,32650.0,37786.0,0.84,21.01,20.09,45.32,198.0,0.86,1.53,27.98,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.00,-810.92,H-SC,100.00,67753.0,-27563.0,72259.0,1.02,-28.92,106.65,46.89,51.0,-0.38,0.55,33.63,X40N,NTT,MISC
58,QUESS,711.66,986.00,NaN,NaN,98.94,79793.0,-76061.0,136143.0,2.63,-48.80,170.62,38.55,NaN,-0.56,0.65,22.18,XY24,NTT,MISC
79,TATAELXSI,7332.28,9000.00,-20.18,X-MC,97.87,83846.0,-18806.0,42158.0,-1.82,-18.32,50.28,22.74,33.0,-0.45,0.68,26.03,X40N,NTT,IT
1,AARTIIND,487.04,919.00,35.69,M-SC,96.81,77847.0,-3489.0,75628.0,-0.52,-4.29,97.15,88.69,118.0,-0.05,0.64,31.27,XR,NTT,CHEMICALS
78,TANLA,917.30,1963.11,-41.41,H-SC,95.74,143302.0,-96113.0,369074.0,11.04,-40.15,257.55,114.01,29.0,-0.26,1.17,31.43,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.32
1,25,41.01
2,50,69.53


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.76
X40     14.50
X40N    13.98
AR      13.86
XR      10.36
XY25    10.27
X200     1.21
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.25
X-LC    18.31
M-SC    16.60
M-LC     8.44
H-MC     7.88
X-MC     6.79
H-LC     5.69
M-MC     5.50
X-SC     2.81
L-SC     2.27
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.81,-2.42,35.00
BANKS,11.12,-10.52,64.72
IT,9.28,-21.72,89.64
FINANCE,7.43,-21.67,76.58
MISC,7.20,-28.32,83.53
INSURANCE,6.11,-3.15,34.24
PAINTS,5.59,-19.97,55.47
ELECTRICAL,4.12,-10.23,66.39
REFINERIES,3.66,9.26,7.46


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3091933.0,27
XR,1278110.0,15
AR,1259472.0,15
X40N,1218167.0,17
X40,816776.0,12
SR,209799.0,2
XY25,151487.0,5
X200,70121.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2463772.0,22
M-SC,1974551.0,20
X-LC,739491.0,12
M-MC,594380.0,5
H-MC,430481.0,7
X-SC,365082.0,4
M-LC,345516.0,6
X-MC,321880.0,6
L-SC,320186.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      942309.0      6
M-SC       XY24      778323.0      6
H-SC       X40N      567254.0      7
           AR        545383.0      3
M-SC       XR        435549.0      5
M-MC       XY24      419937.0      3
X-LC       X40       380014.0      5
M-SC       X40N      377661.0      5
H-SC       XR        306392.0      4
M-LC       XY24      224893.0      4
M-SC       X40       202326.0      2
H-LC       AR        193843.0      4
X-LC       AR        189418.0      3
H-MC       XY24      185147.0      2
L-SC       XR        164364.0      2
X-MC       XY24      140355.0      2
X-SC       SR        123379.0      1
           XR        122132.0      1
           X40N      119571.0      2
X-MC       X40N      110870.0      2
M-LC       XR        104372.0      1
H-SC       X40       102434.0      2
X-LC       XY25       99938.0      3
M-SC       AR         94272.0      1
L-SC       XY24       89584.0      1
M-MC       XR         87647.0      1
           AR         86796.0      1
M-SC       SR         86420.0      1
H-MC       AR         73610.0      1
           X40        71259.0      2
X-LC       X200       70121.0      1
L-SC       AR         66238.0      1
X-MC       X40        60743.0      1
H-MC       XR         57654.0      1
           X40N       42811.0      1
H-LC       XY25       35298.0      1
M-LC       XY25       16251.0      1
X-MC       AR          9912.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
